In [1]:
from PIL import Image, ImageDraw, ImageFont

## **Stitches library and symbols**

In [2]:
directory_path = 'C:\\Users\\kwojd\\AppData\\Roaming\\jupyter\\runtime\\My_projects\\GitHub_projects\\KnittingChartGenerator\\Stitches\\'

edge = Image.open(directory_path + 'edge.png')
purl = Image.open(directory_path + 'purl.png')
knit = Image.open(directory_path + 'knit.png')
k2tog = Image.open(directory_path + 'k2tog.png')
p2tog = Image.open(directory_path + 'p2tog.png')
ssk = Image.open(directory_path + 'ssk.png')
ssp = Image.open(directory_path + 'ssp.png')

front_pp = Image.open(directory_path + 'front_pp.png')
front_pk = Image.open(directory_path + 'front_pk.png')
front_kp = Image.open(directory_path + 'front_kp.png')
front_kk = Image.open(directory_path + 'front_kk.png')
back_pp = Image.open(directory_path + 'back_pp.png')
back_pk = Image.open(directory_path + 'back_pk.png')
back_kp = Image.open(directory_path + 'back_kp.png')
back_kk = Image.open(directory_path + 'back_kk.png')

yo = Image.open(directory_path + 'yarnover.png')
M1R = Image.open(directory_path + 'M1R.png')
M1L = Image.open(directory_path + 'M1L.png')


# Stitch library
stitch_library = {
    'p': purl,
    'k': knit,
    'e': edge,
    'k2tog': k2tog,
    'p2tog': p2tog,
    'ssk': ssk,
    'ssp': ssp,
    'front_pp': front_pp,
    'front_pk': front_pk,
    'front_kp': front_kp,
    'front_kk': front_kk,
    'back_pp': back_pp,
    'back_pk': back_pk,
    'back_kp': back_kp,
    'back_kk': back_kk,
    'yo': yo,
    'M1R': M1R,
    'M1L': M1L
}


arrow = Image.open(directory_path + 'arrow.png')
arrow = arrow.resize((int(arrow.size[0]*0.1), int(arrow.size[1]*0.1)))
upper_arrow = arrow.rotate(90, expand=True)

right_arrow = Image.open(directory_path + 'arrow_horizontal.png')
right_arrow = right_arrow.resize((int(right_arrow.size[0]*0.075), int(right_arrow.size[1]*0.075)))
left_arrow = right_arrow.rotate(180, expand=True)

width0, height0 = edge.size
stitch = width0
stitch_double = front_pp.size[0]



## **Script for reading stitches from a file**

In [30]:
Pattern = {}

stitches_per_row = int(input("Enter the number of stitches per row (including edges!): "))
num_of_rows = int(input("Enter the number of rows: "))
chart_name = str(input('Enter the name of your chart: '))
file_path = 'pattern.txt'


def extract_stitch_info(stitch_info):
    stitch_count = ''
    for char in reversed(stitch_info):
        if char.isdigit():
            stitch_count = char + stitch_count
        else:
            break
    stitch_type = stitch_info[:len(stitch_info) - len(stitch_count)]
    return stitch_type, int(stitch_count) if stitch_count else 1


with open(file_path, 'r') as file:
    for i, line in enumerate(file, start=1):
        key = f'Row_{i}'
        stitches_info = line.strip().split(', ')

        stitches = []
        for stitch_info in stitches_info:
            stitch_type, stitch_count = extract_stitch_info(stitch_info)
        
            stitches.extend([stitch_type] * stitch_count)

        Pattern[key] = stitches

print(Pattern)


## **Creating the chart**

In [31]:
background = Image.new("RGB", ((stitches_per_row + 4)*stitch, (num_of_rows + 2)*stitch), (255, 255, 255))
width, height = background.size

x_filling = stitch + 0.93*stitch*(stitches_per_row - 1)
x_margin = round((width - x_filling)/2)
y_filling = stitch + 0.93*stitch*(num_of_rows - 1)
y_margin = round((height - y_filling)/2)

draw = ImageDraw.Draw(background)

# Iterate through each row in the pattern dictionary
for row_index, row_key in enumerate(Pattern):
    row = Pattern[row_key]
    
    # Iterate through each stitch in the row
    double_stitch_count = 0
    for stitch_index, stitch_type in enumerate(row):
        stitch_image = stitch_library[stitch_type]  
             
        # Adjust x_position based on stitch width
        if stitch_type in {'front_kk', 'front_kp', 'front_pk', 'front_pp', 'back_kk', 'back_kp', 'back_pk', 'back_pp'}:
            double_stitch_count += 1
            x_position = x_margin + ((stitches_per_row - stitch_index - 1 - double_stitch_count) * round(0.93*stitch))
        else:
            x_position = x_margin + (stitches_per_row - stitch_index - 1 - double_stitch_count) * (round(0.93*stitch))
        
        y_position = y_margin + (num_of_rows - row_index - 1) * round(0.93*stitch)

        background.paste(stitch_image, (x_position, y_position))
        
        font_type = ImageFont.truetype("times.ttf", 60)
        if row_index % 2 == 0:
            l_text_position = (int(x_margin + x_filling) + 115, 20 + y_margin + (num_of_rows - row_index - 1) * round(0.93*stitch))
            draw.text(l_text_position, f"{row_index + 1}", fill=(0, 0, 0), font=font_type)
            r_arrow_position = (int(x_margin + x_filling) + 30, y_margin + int(stitch/2) - int(left_arrow.size[1]/2) + (num_of_rows - row_index - 1) * round(0.93*stitch))
            background.paste(left_arrow, r_arrow_position)
        elif row_index % 2 == 1:
            l_arrow_position = (x_margin - 110, y_margin + int(stitch/2) - int(left_arrow.size[1]/2) + (num_of_rows - row_index - 1) * round(0.93*stitch))
            background.paste(right_arrow, l_arrow_position)
    
    for n_stitch in range(0, stitches_per_row - 2):
        draw.text((25 + x_margin + stitch + (stitches_per_row - 2 - n_stitch - 1)*(int(0.93*stitch)), y_margin - 75), f"{n_stitch + 1}", fill=(0, 0, 0), font=ImageFont.truetype("times.ttf", 45))
        
report = str(input('Do you want to put a repeat section? (y/n) ' ))
if report == 'y':
    upper_position = input('Where? Type numbers of first and last column (e.g. 3, 12) ')
    start_column, end_column = map(int, upper_position.split(', '))
    upper_l_arrow_position_1 = (x_margin - int(upper_arrow.size[0]/2) - 4 + stitch + (stitches_per_row - start_column - 1) * round(0.93 * stitch), height - 100)
    upper_l_arrow_position_2 = (x_margin - int(upper_arrow.size[0]/2) - 4 + stitch + (stitches_per_row - end_column - 1 - 1) * round(0.93 * stitch), height - 100)
    
    # Calculate the positions for the vertical lines
    line_start_position_1 = (x_margin - 3 + stitch + (stitches_per_row - start_column - 1) * round(0.93 * stitch), y_margin)
    line_end_position_1 = (x_margin - 3 + stitch + (stitches_per_row - start_column - 1) * round(0.93 * stitch), y_margin + y_filling - 5)
    line_start_position_2 = (x_margin - 3 + stitch + (stitches_per_row - end_column - 1 - 1) * round(0.93 * stitch), y_margin)
    line_end_position_2 = (x_margin - 3 + stitch + (stitches_per_row - end_column - 1 - 1) * round(0.93 * stitch), y_margin + y_filling - 5)

    # Draw two vertical lines on the background image
    draw = ImageDraw.Draw(background)
    draw.line([line_start_position_1, line_end_position_1], fill=(0, 0, 0), width=15)
    draw.line([line_start_position_2, line_end_position_2], fill=(0, 0, 0), width=15)

    background.paste(upper_arrow, upper_l_arrow_position_1)
    background.paste(upper_arrow, upper_l_arrow_position_2)
else:
    pass
        
background.save(chart_name + '.png')
background.show()

## **Colouring the chart** 

In [39]:
from PIL import Image

img = Image.open(chart_name + '.png')
# img = Image.open(chart_name + '_colour' + '.png')   # In case we want to provide another colour
replacement_color = (240, 52, 194)

img = img.convert('RGB')
width, height = img.size

file_ranges = 'ranges.txt'

with open(file_ranges, 'r') as file:
    for row_index, line in enumerate(file, start=1):
        print(row_index)
        
        if not line.strip():
            continue
        color_ranges = [range(int(start), int(end) + 1) for start, end in (pair.split('-') for pair in line.strip().split(', '))]
        for range_i in color_ranges:
            for x in range((x_margin + stitch + round(0.93 * stitch) * (stitches_per_row - 2 - (range_i[-1]))), (x_margin + stitch + round(0.93 * stitch) * (stitches_per_row - 2 - (range_i[0] - 1)))):
                for y in range(int(y_margin + (num_of_rows - row_index - 0) * round(0.93 * stitch)),
                               int(y_margin + (num_of_rows - row_index + 1) * round(0.93 * stitch))):
                    r, g, b = img.getpixel((x, y))
                    if r >= 200 and g >= 200 and b >= 200:
                        img.putpixel((x, y), replacement_color)
# Save the modified image
img.save(chart_name + '_colour' + '.png')
